In [0]:
%matplotlib inline

debug = False

In [69]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
SOS_token = 0
EOS_token = 1
UNK_token = 2
PAD_token = 3
# MAX_LENGTH = 1000

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2:"UNK"}
        self.n_words = 3  # Count SOS and EOS and UNK

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def Reverse(lst): 
    return [ele for ele in reversed(lst)]

In [0]:
def readLangs(lang1, lang2, reverse=False, model="dev"):
    
    if model == "dev":
        source = "/content/gdrive/My Drive/NLPA/NLA S20 - Assignment 2 Data/enghin/dev.en"
        target = "/content/gdrive/My Drive/NLPA/NLA S20 - Assignment 2 Data/enghin/dev.hi"
    else if model == "test":
        source = "/content/gdrive/My Drive/NLPA/NLA S20 - Assignment 2 Data/enghin/test.en"
        target = "/content/gdrive/My Drive/NLPA/NLA S20 - Assignment 2 Data/enghin/test.hi"
    else:
        source = "/content/gdrive/My Drive/NLPA/NLA S20 - Assignment 2 Data/enghin/train.en"
        target = "/content/gdrive/My Drive/NLPA/NLA S20 - Assignment 2 Data/enghin/train.hi"
    
    eng_lines = open(source, encoding='utf-8').read().strip().split('\n')
    hin_lines = open(target, encoding='utf-8').read().strip().split('\n')

    lines = []
    for i in range(len(eng_lines)):
        lines.append(eng_lines[i] + '\t' + hin_lines[i])

    # Split every line into pairs and normalize
    # pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    pairs = [[s for s in l.split('\t')] for l in lines]
    #print(pairs)
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [0]:
MAX_LENGTH = 20
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [0]:
def prepareData(lang1, lang2, reverse=False, model="dev", filter_sentence=False):
    
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse, model)
    #print(input_lang, output_lang, pairs)
    if(filter_sentence):
        pairs = filterPairs(pairs)
    
    print("Read %s sentence pairs" % len(pairs))
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    
    return input_lang, output_lang, pairs

In [76]:
input_lang, output_lang, pairs = prepareData('eng', 'hi', reverse=False, model="dev", filter_sentence=True)
print(random.choice(pairs))
orig_pairs = []
for p in pairs:
    temp = str(p[0]).split(' ')
    temp = Reverse(temp)
    listToStr = ' '.join([str(elem) for elem in temp]) 
#     print("listToStr")
#     print(listToStr)
    listToStr = str(listToStr)
    orig_pairs.append(listToStr+'\t'+str(p[1]))
#     print(orig_pairs[-1])
print("orig_pairs")
print(orig_pairs[0])
print(random.choice(orig_pairs))

Read 256 sentence pairs
Trimmed to 256 sentence pairs
Counting words...
Counted words:
eng 1117
hi 1170
['Fever does not go down , diarrhoea does not stop , weight decreases .', 'बुखार नहीं जाता , डायरिया नहीं रूकता , वजन कम हो जाता है ।']
orig_pairs
. diseases eye from safe be also will you but	बल्कि आप नेत्ररोगों से भी बचे रहेंगे ।
. relations sex unprotected through spreads AIDS	असुरक्षित यौन संबंध से एड्स फैलता है ।


The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.





In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers, bidirectional=True, dropout=dropout) # batch_first=False,
        # self.gru = nn.GRU(self.hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedding = self.embedding(input).view(1, 1, -1)
        output = embedding
        output, hidden = self.gru(output, hidden)
        return output, hidden
        # packed = pack_padded_sequence(x, lengths, batch_first=True)
        # output, final = self.gru(packed)
        # output, _ = pad_packed_sequence(output, batch_first=True)

        # Manually concatenating the final states for both directions
        # fwd_final = final[0:final.size(0):2]
        # bwd_final = final[1:final.size(0):2]
        # final = torch.cat([fwd_final, bwd_final], dim=2)  # [num_layers, batch, 2*dim]

        # return output, final

    def initHidden(self):
        return torch.zeros(self.num_layers*2, 1, self.hidden_size, device=device)

The Attention Decoder
---------------------





In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, num_layers=1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.num_layers = num_layers
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size*2)

        # self.pre_output_layer = nn.Linear(hidden_size + 2*hidden_size + emb_size, hidden_size, bias=False)

        # Attention -- general/bilinear
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 4, self.hidden_size)
        self.attn_general = nn.Linear(self.max_length, self.max_length)
        self.lcl_wa_into_hs = nn.Linear(self.hidden_size*2, self.hidden_size*2)

        # self.gru = nn.GRU(emb_size + 2*hidden_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, self.num_layers, bidirectional=True)
        self.out = nn.Linear(self.hidden_size*2, self.output_size)

        # self.attn_coverage = nn.Linear(self.max_length, self.hidden_size)
        # self.attn_coverage_cat = nn.Linear(self.hidden_size*3, self.max_length)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
       
        alphas = self.attn_general(torch.matmul(self.lcl_wa_into_hs(embedded[0]), encoder_outputs.T))
        
        attn_weights = F.softmax(alphas, dim = 1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(self.num_layers*2, 1, self.hidden_size, device=device)

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] if word in lang.word2index else UNK_token for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf>`__.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [0]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size*2, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden
    # decoder_hidden = encoder_outputs

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            # For Paper 2 
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            # For Paper 2 
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
loss_list = []
epoch_list = []

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [0]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            
            loss_list.append(print_loss_avg)
            epoch_list.append(iter)
            print('epoch = ',epoch_list[-1],'  loss = ',loss_list[-1])

            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [0]:
def showPlot(loss_list, epoch_list):
    plt.plot(epoch_list, loss_list)
    plt.xticks(np.arange(0, 75000, 10000)) 
    plt.yticks(np.arange(0, 5, 0.5)) 
    plt.savefig("test.png")
    plt.show()
    plt.close('all')

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size*2, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            try:
              if topi.item() == UNK_token:
                  decoded_words.append('<UNK>')
              if topi.item() == EOS_token:
                  decoded_words.append('<EOS>')
                  break
              else:
                  decoded_words.append(output_lang.index2word[topi.item()])
            except:
              continue
            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [0]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(orig_pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [87]:
print(output_lang.n_words)
print(input_lang.n_words)
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, attn_decoder1, 25000, print_every=5000)

1170
1117


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


epoch =  5000   loss =  3.259508546816716
4m 46s (- 19m 5s) (5000 20%) 3.2595
epoch =  10000   loss =  0.17473994010808072
9m 49s (- 14m 43s) (10000 40%) 0.1747
epoch =  15000   loss =  0.011075238883394704
14m 48s (- 9m 52s) (15000 60%) 0.0111
epoch =  20000   loss =  0.005814625028227991
19m 44s (- 4m 56s) (20000 80%) 0.0058
epoch =  25000   loss =  0.004146413236024421
24m 40s (- 0m 0s) (25000 100%) 0.0041


Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [0]:
no_of_epoch = 25000
no_hidden_states = 256
# model_type = "AttnDecoder"
# model_name_dec =  model_type+"_Model_"+str(no_of_epoch)+"_"+str(no_hidden_states)
# torch.save(attn_decoder1.state_dict(), model_name_dec)
# # device = torch.device('cpu')
# decoder_model = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)
# decoder_model.load_state_dict(torch.load(model_name_dec, map_location=device))

In [0]:
datafile = "dev_"
# epochs
task = "BiGRU_"
model_name = task+datafile+str(no_of_epoch)+"_"+str(no_hidden_states)+".encoder"

torch.save(encoder1.state_dict(), model_name)
device = torch.device('cpu')
# encoder_model = EncoderRNN(input_lang.n_words, hidden_size).to(device)
# encoder_model.load_state_dict(torch.load(model_name, map_location=device))


model_name = task+datafile+str(no_of_epoch)+"_"+str(no_hidden_states)+".attndecoder"
torch.save(attn_decoder1.state_dict(), model_name)
device = torch.device('cpu')
# decoder_model = DecoderRNN(hidden_size, output_lang.n_words).to(device)
# decoder_model.load_state_dict(torch.load(model_name, map_location=device))

# showPlot(plot_losses, plot_epoch)

In [95]:
device = torch.device('cuda')
evaluateRandomly(encoder1, attn_decoder1)

> .
=  
< यहाँ से थोड़ा और नीचे बढ़ने पर पहाड़ियों के लिए । <EOS>

> .
=  
< यहाँ से थोड़ा और नीचे बढ़ने पर पहाड़ियों से घिरा गंगामाया मिला । <EOS>

> .
=  
< यहाँ से थोड़ा और नीचे बढ़ने पर पहाड़ियों का इरादा किया । <EOS>

> .
=  
< यहाँ से थोड़ा और नीचे बढ़ने पर पहाड़ियों से घिरा गंगामाया मिला । <EOS>

> .
=  
< यहाँ से थोड़ा और नीचे बढ़ने पर पहाड़ियों के लिए । <EOS>

> .
=  
< यहाँ से थोड़ा और नीचे बढ़ने पर पहाड़ियों से घिरा गंगामाया मिला । <EOS>

> .
=  
< यहाँ से थोड़ा और नीचे बढ़ने पर निश्‍चित ही विजय प्राप्त की जा सकती है । <EOS>

> .
=  
< यहाँ से थोड़ा और नीचे बढ़ने पर पहाड़ियों से घिरा गंगामाया मिला । <EOS>

> .
=  
< यहाँ से थोड़ा और नीचे बढ़ने पर पहाड़ियों के लिए । <EOS>

> .
=  
< यहाँ से थोड़ा और नीचे बढ़ने पर पहाड़ियों के लिए । <EOS>



In [96]:
print(epoch_list)
print(loss_list)

[5000, 10000, 15000, 20000, 25000]
[3.259508546816716, 0.17473994010808072, 0.011075238883394704, 0.005814625028227991, 0.004146413236024421]


In [0]:
s = "epoch_list_"+model_name_dec+'.pkl' 
with open(s, 'wb') as f:
    pickle.dump(epoch_list, f)

s = "loss_list"+model_name_dec+'.pkl' 
with open(s, 'wb') as f:
    pickle.dump(loss_list, f)

In [98]:
print("after loading pickles")
s = "epoch_list_"+model_name_dec+'.pkl' 
with open(s, 'wb') as f:
    mynewlist = pickle.load(f)
    print(mynewlist)

s = "loss_list"+model_name_dec+'.pkl' 
with open(s, 'wb') as f:
    mynewlist = pickle.load(f)
    print(mynewlist)

after loading pickles


UnsupportedOperation: ignored

In [0]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction    

def calculate_bleu(pred_trg, real_trg):
    smoothie = SmoothingFunction().method4
    score = sentence_bleu(real_trg, pred_trg, smoothing_function=smoothie)
    return score 

def calculate_Result(encoder, decoder,lcl_pairs, n=50):
    device = torch.device('cuda')

    result_value_bleu_score = []
    
    for i in range(n):
        pair = random.choice(lcl_pairs)
        if debug:
          print('>', pair[0])
          print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        if debug:
          print('<', output_sentence)
        reference = [pair[1].split()]
        if debug:
          print('--', reference)
        output_words = output_words[:-1]
        temp  = []
        for ow in output_words:
          if ow!='':
            temp.append(ow)
        output_words = temp
        target_predicted = output_words
        
        if debug:        
          print('<<', output_words)
        
        score = calculate_bleu(target_predicted,reference)
        
        if debug:
          print("---Value",score)
        
        result_value_bleu_score.append((pair[0],pair[1].split(),target_predicted,score))

    return result_value_bleu_score

In [105]:
input_lang, output_lang, pairs = prepareData('eng', 'hi', reverse=False, model="test", filter_sentence=True)
print(random.choice(pairs))

Read 30810 sentence pairs
Trimmed to 30810 sentence pairs
Counting words...
Counted words:
eng 23989
hi 27143
['With this one will get rid of itching and the skin will become soft and radiant as well .', 'अगर आप खाज - खुजली आदि से परेशान हैं तो कच्चे पपीते का दूध निकालकर लगाएँ ।']


In [106]:
global debug
debug = 1
device = torch.device('cuda')
result_value_bleu_score = calculate_Result(encoder1, attn_decoder1, pairs)
result_value_bleu_score_dict = {}
result_value_bleu_score_dict['result'] = result_value_bleu_score 
# torch.save(result_value_bleu_score_dict, train_result_data_path)
for item in result_value_bleu_score:
  if (item[3]>0):
    print(" Source Language ",item[0])
    print(" Input Target",item[1])
    print(" Output Target",item[2])
    print(" Score ",item[3])

> First the coastal journey of Chandra river and then the Bhaga river .
= पहले चंद्रनदी का तटीय सफर , फिर भागा नदी का ।
< करें आरम्भिक आसानी मसूड़ों टाँके होठों जाता सम्भव <EOS>
-- [['पहले', 'चंद्रनदी', 'का', 'तटीय', 'सफर', ',', 'फिर', 'भागा', 'नदी', 'का', '।']]
<< ['करें', 'आरम्भिक', 'आसानी', 'मसूड़ों', 'टाँके', 'होठों', 'जाता', 'सम्भव']
---Value 0
> Yamuna is joining Ganga with a lot of love .
= बहुत प्रेम से यमुना गंगा में समा जा रही है ।
< होती गये कदम दबाव उद्देश्य बाद प्राथमिक सम्भव <EOS>
-- [['बहुत', 'प्रेम', 'से', 'यमुना', 'गंगा', 'में', 'समा', 'जा', 'रही', 'है', '।']]
<< ['होती', 'गये', 'कदम', 'दबाव', 'उद्देश्य', 'बाद', 'प्राथमिक', 'सम्भव']
---Value 0
> On leaving Rangiya there are fields of wheat on both sides of the road .
= रंगिया छोड़ते ही सड़क के दोनों ओर धान के खेत हैं ।
< होती गये वाला कुष्ठ निकट जीवाणु आँखों कैपस्यूलर छूत हो माइकोबेक्टीरियम वाला लैप्री मुख्यतया क्षमता सीमित सम्भव <EOS>
-- [['रंगिया', 'छोड़ते', 'ही', 'सड़क', 'के', 'दोनों', 'ओर', 'धान', 'के', 'खेत', 'हैं',